In [7]:
import pandas as pd
import os
import numpy as np
import sys
import tsfresh as tsf
from tsfresh.utilities.dataframe_functions import impute

In [ ]:
"""
Create TSFRESH Time series features

"""


win_lengths = [10] # hyperparameter

# drop all categorical columns

col_list = ['log_wap1_log_price_ret_vol', 'Wap1_log_price_ret_per_spread_sqr_vol', 'log_1e2_sad_ask_price1_stnd','log_1e2_sad_bid_price1_stnd'
            ,'log1p_sad_order_count1_stnd','exp_log_liq1_ret_*_wap_eqi_price1_ret_vol','log_liq2_ret_*_wap_eqi_price1_ret_vol_15_ratio_median_stock',
            'wap2_logprice_ret_changes_n_wap1_logprice_ret_constant_sqr_vol','wap1_log_price_ret_pos-neg_log_liq_ret_sqr_vol','wap1_log_price_ret_normalized*mean_centered_per_wap1_lprice_ret_vol_:0'
            ,'wap1_lprice_ret_vol_ati_*_wap1_lprice_ret_vol_ast_per_wap1_lprice_ret_vol_:20','soft_stock_mean_tvpl2_:0','soft_stock_mean_tvpl2_liqf_volf20',
'v1proj_25_15_lr1_high_corr_stocks','v1proj_25_15_vol1_high_corr_stocks','lsvol','liqvol1','root_book_delta_count','v1proj_29_q1','v1proj_29_15_q3','v1proj_25_std',
'tvpl2_rmed2v1',
'tvpl2',
'tvpl2_smean_vol',


'v1liq2projt10',

'liqt10rf29',

'v1liq2sprojt10f25',


'v1spprojt15f25',


'v1spprojt15f29_q3',
'v1spprojt15f25_q1',

'v1spprojtf29_q1',
'v1spprojtf25_q3',

'wap1_log_price_ret_volstock_mean_from_10',
'wap1_log_price_ret_vol_from_20',

'log_+1e-3_trade_price_real_vol_stnd','log1p_log_1ptrade_size_std_stnd_stnd',
'log1p_trade_order_count_std_stnd',
'exp_bs_bp_corr1_stnd','bp_as_corr1','log1p_range_order_count1_stnd','size_order_count_corr1','log_book_ewma_vol_stnd','log1p_trade_ewma_vol_stnd',
'arctanh_min_1e-3_bp_ap_corr2_stnd','log_wap1_lprice_ret_vol_ati_*_wap1_lprice_ret_avg_ast_per_wap1_lprice_ret_vol_:0_stnd','exp_root_trade_count_var_stnd'
]


feature_settings = {'fft_coefficient': [{'coeff': 0, 'attr': 'real'}, {'coeff': 1, 'attr': 'real'}],
                    #'mean': None,
                    #'standard_deviation': None,
                    'skewness': None,
                    'kurtosis': None,
                    #'minimum': None,
                    #'maximum': None,
                    #'sum_values': None,
                    #'variance': None,
                    #'absolute_sum_of_changes': None,
                    'count_above_mean': None,
                    'count_below_mean': None,
                    #'longest_strike_above_mean': None,
                    #'longest_strike_below_mean': None,
                    #'number_crossing_m': [{'m': 0}],
                    'autocorrelation': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
                    'partial_autocorrelation': [{'lag': 1}, {'lag': 2}, {'lag': 3}]}


for c in col_list[8:]: # select 1 numeric column at a time, leave out target and long lags
    print('time series name: ',c)
    for win in win_lengths:
        num_df = raw_feat_df.loc[:,['datetime','group',c]]
        print('num_df.shape', num_df.shape)
        df_rolled = tsf.utilities.dataframe_functions.roll_time_series(num_df,column_id='group',column_sort='datetime',max_timeshift=win)
        df_rolled.drop(columns=['group'], inplace=True)
        df_features = tsf.extract_features(df_rolled, column_id='id', column_sort='datetime', 
                                            default_fc_parameters=feature_settings)
        del df_rolled # free-up the RAM

        # Remove any missing values
        imputed_features = impute(df_features)
        del df_features # free-up the RAM

        # Select a subset of relevant features using tsfresh's built-in feature selection function
        raw_feat_df.index= range(raw_feat_df.shape[0])
        imputed_features.index = range(raw_feat_df.shape[0])
        selected_features = tsf.select_features(imputed_features, raw_feat_df['target'])
        del imputed_features # free-up the RAM

        # Print the names of the selected features
        print('selected_features.columns',selected_features.columns)

        print('selected_features.columns: ',selected_features.columns)
        print('selected_features.shape: ',selected_features.shape)

        print('all_stns_df.columns: ',raw_feat_df.columns)
        print('all_stns_df.shape: ',raw_feat_df.shape)



        raw_feat_df = pd.concat([raw_feat_df,selected_features],axis=1)



        del num_df # free-up the RAM
        del selected_features # free-up the RAM


raw_feat_df.shape